In [1]:

import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch import nn, Tensor
import torch.nn.init as init
from typing import Callable, Tuple
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.interpolate import griddata
import math

_scaling_min = 0.001

torch.set_default_dtype(torch.float64)

def unsqueeze(x):
    return x.unsqueeze(0).unsqueeze(-1).detach()

class ActNorm(torch.nn.Module):
    """
    ActNorm layer
    """
    
    def __init__(self,
                 num_features: int, # number of input dimension
                 logscale_factor: float = 1.,
                 scale: float = 1.,
                 learn_scale: bool = True
                 ) -> None:
        super(ActNorm, self).__init__()

        self.initialized = False
        self.num_features = num_features

        self.register_parameter('b', nn.Parameter(torch.zeros(1, num_features, 1), requires_grad=True))
        self.learn_scale = learn_scale
        if learn_scale:
            self.logscale_factor = logscale_factor
            self.scale = scale
            self.register_parameter('logs', nn.Parameter(torch.zeros(1, num_features, 1), requires_grad=True))
            
    def forward_transform(self, x, logdet=0):
        input_shape = x.size()
        x = x.view(input_shape[0], input_shape[1], -1)

        if not self.initialized:
            self.initialized = True
            
            sum_size = x.size(0) * x.size(-1)
            b = -torch.sum(x, dim=(0, -1)) / sum_size
            self.b.data.copy_(unsqueeze(b).data)

            if self.learn_scale:
                var = unsqueeze(torch.sum((x + unsqueeze(b)) ** 2, dim=(0, -1)) / sum_size)
                logs = torch.log(self.scale / (torch.sqrt(var) + 1e-6)) / self.logscale_factor
                self.logs.data.copy_(logs.data)

        b = self.b
        output = x + b

        if self.learn_scale:
            logs = self.logs * self.logscale_factor
            scale = torch.exp(logs) + _scaling_min
            output = output * scale
            dlogdet = torch.sum(torch.log(scale)) * x.size(-1)  # c x h

            return output.view(input_shape), logdet + dlogdet
        else:
            return output.view(input_shape), logdet

class ActNormNoLogdet(ActNorm):

    def forward(self, x):
        return super(ActNormNoLogdet, self).forward_transform(x)[0]

class SequentialFlow(torch.nn.Module):

    def __init__(self, flows):
        super(SequentialFlow, self).__init__()
        self.flows = torch.nn.ModuleList(flows)

    def forward_transform(self, x, logdet=0):
        for flow in self.flows:
            x, logdet = flow.forward_transform(x, logdet)
        return x, logdet



def symm_softplus(x, softplus_=torch.nn.functional.softplus):
    return softplus_(x) - 0.5 * x


def softplus(x):
    return nn.functional.softplus(x)


def gaussian_softplus(x):
    z = np.sqrt(np.pi / 2)
    return (z * x * torch.erf(x / np.sqrt(2)) + torch.exp(-x**2 / 2) + z * x) / (2*z)


def gaussian_softplus2(x):
    z = np.sqrt(np.pi / 2)
    return (z * x * torch.erf(x / np.sqrt(2)) + torch.exp(-x**2 / 2) + z * x) / z


def laplace_softplus(x):
    return torch.relu(x) + torch.exp(-torch.abs(x)) / 2


def cauchy_softplus(x):
    # (Pi y + 2 y ArcTan[y] - Log[1 + y ^ 2]) / (2 Pi)
    pi = np.pi
    return (x * pi - torch.log(x**2 + 1) + 2 * x * torch.atan(x)) / (2*pi)


def activation_shifting(activation):
    def shifted_activation(x):
        return activation(x) - activation(torch.zeros_like(x))
    return shifted_activation


def get_softplus(softplus_type='softplus', zero_softplus=False):
    if softplus_type == 'softplus':
        act = nn.functional.softplus
    elif softplus_type == 'gaussian_softplus':
        act = gaussian_softplus
    elif softplus_type == 'gaussian_softplus2':
        act = gaussian_softplus2
    elif softplus_type == 'laplace_softplus':
        act = gaussian_softplus
    elif softplus_type == 'cauchy_softplus':
        act = cauchy_softplus
    else:
        raise NotImplementedError(f'softplus type {softplus_type} not supported.')
    if zero_softplus:
        act = activation_shifting(act)
    return act


class Softplus(nn.Module):
    def __init__(self, softplus_type='softplus', zero_softplus=False):
        super(Softplus, self).__init__()
        self.softplus_type = softplus_type
        self.zero_softplus = zero_softplus

    def forward(self, x):
        return get_softplus(self.softplus_type, self.zero_softplus)(x)


class SymmSoftplus(torch.nn.Module):

    def forward(self, x):
        return symm_softplus(x)


class PosLinear(torch.nn.Linear):
    def forward(self, x: Tensor) -> Tensor:
        gain = 1 / x.size(1)
        return nn.functional.linear(x, torch.nn.functional.softplus(self.weight), self.bias) * gain


class PosLinear2(torch.nn.Linear):
    def forward(self, x: Tensor) -> Tensor:
        return nn.functional.linear(x, torch.nn.functional.softmax(self.weight, 1), self.bias)


class PosConv2d(torch.nn.Conv2d):

    def reset_parameters(self) -> None:
        super().reset_parameters()
        # noinspection PyProtectedMember,PyAttributeOutsideInit
        self.fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)

    def forward(self, x: Tensor) -> Tensor:
        return self._conv_forward(x, torch.nn.functional.softplus(self.weight)) / self.fan_in



class ICNN(torch.nn.Module):
    def __init__(self, dim=2, dimh=16, num_hidden_layers=1):
        super(ICNN, self).__init__()

        Wzs = list()
        Wzs.append(nn.Linear(dim, dimh))
        for _ in range(num_hidden_layers - 1):
            Wzs.append(PosLinear(dimh, dimh, bias=False))
        Wzs.append(PosLinear(dimh, 1, bias=False))
        self.Wzs = torch.nn.ModuleList(Wzs)

        Wxs = list()
        for _ in range(num_hidden_layers - 1):
            Wxs.append(nn.Linear(dim, dimh))
        Wxs.append(nn.Linear(dim, 1, bias=False))
        self.Wxs = torch.nn.ModuleList(Wxs)
        self.act = nn.Softplus()

    def forward(self, x):
        z = self.act(self.Wzs[0](x))
        for Wz, Wx in zip(self.Wzs[1:-1], self.Wxs[:-1]):
            z = self.act(Wz(z) + Wx(x))
        return self.Wzs[-1](z) + self.Wxs[-1](x)





class ICNN3(torch.nn.Module):
    def __init__(self, 
                 dim: int = 2, # input dimension 
                 dimh: int = 16, # hidden layer dimension
                 num_hidden_layers: int = 2, # number of hidden layer 
                 symm_act_first: bool = False, # refer to the paper
                 softplus_type: str = 'softplus', # refer to the paper
                 zero_softplus: bool = False # refer to the paper
                 ) -> None:
        super(ICNN3, self).__init__()

        self.act = Softplus(softplus_type=softplus_type, zero_softplus=zero_softplus)
        self.symm_act_first = symm_act_first

        Wzs = list()
        Wzs.append(nn.Linear(dim, dimh))
        for _ in range(num_hidden_layers - 1):
            Wzs.append(PosLinear(dimh, dimh // 2, bias=True))
        Wzs.append(PosLinear(dimh, 1, bias=False))
        self.Wzs = torch.nn.ModuleList(Wzs)

        Wxs = list()
        for _ in range(num_hidden_layers - 1):
            Wxs.append(nn.Linear(dim, dimh // 2))
        Wxs.append(nn.Linear(dim, 1, bias=False))
        self.Wxs = torch.nn.ModuleList(Wxs)

        Wx2s = list()
        for _ in range(num_hidden_layers - 1):
            Wx2s.append(nn.Linear(dim, dimh // 2))
        self.Wx2s = torch.nn.ModuleList(Wx2s)

        actnorms = list()
        for _ in range(num_hidden_layers - 1):
            actnorms.append(ActNormNoLogdet(dimh // 2))
        actnorms.append(ActNormNoLogdet(1))
        actnorms[-1].b.requires_grad_(False)
        self.actnorms = torch.nn.ModuleList(actnorms)

    def forward(self, x):
        if self.symm_act_first:
            z = symm_softplus(self.Wzs[0](x), self.act)
        else:
            z = self.act(self.Wzs[0](x))
        for Wz, Wx, Wx2, actnorm in zip(self.Wzs[1:-1], self.Wxs[:-1], self.Wx2s[:], self.actnorms[:-1]):
            z = self.act(actnorm(Wz(z) + Wx(x)))
            aug = Wx2(x)
            aug = symm_softplus(aug, self.act) if self.symm_act_first else self.act(aug)
            z = torch.cat([z, aug], 1)
        return self.actnorms[-1](self.Wzs[-1](z) + self.Wxs[-1](x))



class DeepConvexFlow(torch.nn.Module):
    def __init__(self, 
                 icnn: Callable[..., Tensor], # ICNN object
                 bias_w1: float = 0.0, # bias parameters in actnorm layer
                 trainable_w0: bool = True # weight parameters in actnorm layer
                 ) -> None:
        super(DeepConvexFlow, self).__init__()

        self.icnn = icnn
        self.w0 = torch.nn.Parameter(torch.log(torch.exp(torch.ones(1)) - 1), requires_grad=trainable_w0)
        self.w1 = torch.nn.Parameter(torch.zeros(1) + bias_w1)

        
    def get_potential(self, 
                      x: Tensor
                      ) -> Tensor:
        """compute the potential F

        Args:
            x (Tensor): coordinates

        Returns:
            Tensor: potential F (N, 1)
        """
        n = x.size(0)
        icnn = self.icnn(x)
        return F.softplus(self.w1) * icnn + F.softplus(self.w0) * (x.view(n, -1) ** 2).sum(1, keepdim=True) / 2
    
    def forward(self, 
                x: Tensor
                ) -> Tensor:
        """forward function, compute the convex potential f, gradient of F

        Args:
            x (Tensor): coordinate

        Returns:
            Tensor: convex potential, f (N, d)
        """
        with torch.enable_grad():
            x = x.clone().requires_grad_(True)
            F = self.get_potential(x)
            f = torch.autograd.grad(F.sum(), x, create_graph=True)[0]
        return f
    
    def forward_transform(self, 
                          x: Tensor, 
                          logdet: Tensor = 0
                          ) -> Tuple[Tensor, Tensor]:
        """compute the convex potential f and log determinant

        Args:
            x (Tensor): coordinate
            logdet (Tensor, optional): log determinant (N, 1). Defaults to 0.

        Returns:
            Tuple[Tensor, Tensor]: convex potential, log determinant
        """
        
        with torch.enable_grad():
            x = x.clone().requires_grad_(True)
            F = self.get_potential(x)
            f = torch.autograd.grad(F.sum(), x, create_graph=True)[0]
            H = []
            
            for i in range(f.shape[1]):
                H.append(torch.autograd.grad(f[:, i].sum(), x, create_graph = True, retain_graph = True)[0])
                
            H = torch.stack(H, dim = 1)
        
        return f, logdet + torch.slogdet(H).logabsdet





class OOMFormatter(ticker.ScalarFormatter):
    def __init__(self, order=0, fformat="%1.1f", offset=True, mathText=True):
        self.oom = order
        self.fformat = fformat
        ticker.ScalarFormatter.__init__(self,useOffset=offset,useMathText=mathText)
    def _set_order_of_magnitude(self):
        self.orderOfMagnitude = self.oom
    def _set_format(self, vmin=None, vmax=None):
        self.format = self.fformat
        if self._useMathText:
             self.format = r'$\mathdefault{%s}$' % self.format

def DensityPlot(x, y, rho, t):
    _, ax = plt.subplots(figsize=(10, 10))
    h = ax.scatter(x, y, c = rho, alpha=1, cmap= 'viridis',  marker='o', s=10)
    ax.grid(color='grey', linestyle='-', linewidth=0.3)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(h, cax=cax, format=OOMFormatter(-1, mathText=False)).ax.yaxis.offsetText.set_fontsize(30)
    plt.tick_params(labelsize=30)
    ax.set_xlim([-1.5, 1.5])
    ax.set_ylim([-1.5, 1.5])
    ax.tick_params(axis='both', which='major', labelsize=30)
    plt.savefig(f'./plots/porus2d{t}.png', pad_inches = 0.05, bbox_inches='tight')
    plt.show()
    plt.close()

def Porus(phi: Tensor,
                 coor: Tensor,
                 rho: Tensor,
                 lgdet: Tensor,
                 alpha: float
                 ) -> Tensor:
    """Loss function for Fokker Planck equation

    Args:
        phi (Tensor): The convex potential of the normalizing flow [N, d]
        coor (Tensor): Input data [N, d]
        rho (Tensor): Estimated function [N, 1] 
        lgdet (Tensor): Log determinant [1, N]
        V (Tensor): density [N, 1]

    Returns:
        loss
    """
    Cap_Phi = 100 * torch.mean(rho*torch.sum((phi - coor)**2, dim = 1, keepdim = True)) # captical phi
    vol = torch.mean(1/(alpha - 1) * (rho / (torch.exp(lgdet)+1e-8))**(alpha) * torch.exp(lgdet)) # second part
    
    return Cap_Phi + vol, vol

def Barenblatt(x: Tensor, 
        t: float
        ) -> Tensor:

    # print(x)

    c0 = 0.1
    alpha = 4
    d = x.size()[1]
    # print(t)
    k = 1/(alpha - 1 + 2/d)

    # print(k)

    val = c0 - k * (alpha - 1) / (t**(2*k/d) * (2 * d * alpha)) * torch.sum(x**2, dim = 1, keepdim = True) 
    return F.relu(val, inplace=True)**(1 / (alpha - 1)) * t**(-k)

def disk_grid_regular (n, r, c, ng ):
    "copy from https://people.sc.fsu.edu/~jburkardt/py_src/disk_grid/disk_grid.py"

    cg = np.zeros ( [ 2, ng ] )

    p = 0

    for j in range ( 0, n + 2 ):

        i = 0
        x = c[0]
        y = c[1] + r * float ( 2 * j ) / float ( 2 * n + 1 )
        cg[0,p] = x
        cg[1,p] = y
        p = p + 1

        if ( 0 < j ):

            cg[0,p] = x
            cg[1,p] = 2.0 * c[1] - y
            p = p + 1

        while ( True ):

            i = i + 1
            x = c[0] + r * float ( 2 * i ) / float ( 2 * n + 1 )
            if ( r * r < ( x - c[0] ) ** 2 + ( y - c[1] ) ** 2 ):
                break

            cg[0,p] = x
            cg[1,p] = y
            p = p + 1
            cg[0,p] = 2.0 * c[0] - x
            cg[1,p] = y
            p = p + 1

            if ( 0 < j ):
                cg[0,p] = x
                cg[1,p] = 2.0 * c[1] - y
                p = p + 1;
                cg[0,p] = 2.0 * c[0] - x
                cg[1,p] = 2.0 * c[1] - y
                p = p + 1
    cg = np.transpose(cg)
    return cg[~np.all(cg == 0, axis=1)]

def disk_grid_regular_count ( n, r, c ):

#*****************************************************************************80
#
## disk_grid_regular_count() counts the grid points inside a disk.
#
#  Discussion:
#
#    The grid is defined by specifying the radius and center of the disk,
#    and the number of subintervals N into which the horizontal radius
#    should be divided.  Thus, a value of N = 2 will result in 5 points
#    along that horizontal line.
#
#  Licensing:
#
#    This code is distributed under the GNU LGPL license.
#
#  Modified:
#
#    07 April 2015
#
#  Author:
#
#    John Burkardt
#
#  Input:
#
#    integer N, the number of subintervals.
#
#    real R, the radius of the disk.
#
#    real C(2), the coordinates of the center of the disk.
#
#  Output:
#
#    integer NG, the number of grid points inside the disk.
#
    ng = 0

    for j in range ( 0, n + 2 ):

        i = 0
        x = c[0]
        y = c[1] + r * float ( 2 * j ) / float ( 2 * n + 1 )
        ng = ng + 1
        if ( 0 < j ):
            ng = ng + 1

        while ( True ):

            i = i + 1
            x = c[0] + r * float ( 2 * i ) / float ( 2 * n + 1 )

            if ( r * r < ( x - c[0] ) ** 2 + ( y - c[1] ) ** 2 ):
                break

            ng = ng + 1
            ng = ng + 1

            if ( 0 < j ):
                ng = ng + 1
                ng = ng + 1
    return ng

In [ ]:

center = torch.tensor([[0.,0.]])
c0 = 0.1
alpha = 4
d = 2
kk = 1/(alpha - 1 + 2/d)
print(kk)
r = 0.1**(kk/d) * math.sqrt(2*d*alpha*c0/(kk*(alpha - 1)))
print(r)
ng = disk_grid_regular_count(50, r - 0.01, (0,0))
coor = disk_grid_regular(50, r - 0.01, (0, 0), ng)
coor = torch.tensor(coor)
coor = torch.cat((coor, center), dim=0)

theta = torch.linspace(0, 2 * torch.pi, 501)

for k in range(1):
  x = (r - k * 0.01) * torch.cos(theta)
  y = (r - k * 0.01) * torch.sin(theta)
  coor1 = torch.cat( (x.flatten()[:, None], y.flatten()[:, None]), dim = 1 )
  coor = torch.cat((coor, coor1), dim = 0)

coor.size()

rho = Barenblatt(coor, 0.1)
DensityPlot(coor.numpy()[:,0], coor.numpy()[:,1], rho.numpy(), 0)

dimx = 2
nblocks = 1
depth = 6
k = 32
symm_act_first = True
zero_softplus = False
softplus_type = 'gaussian_softplus2'



icnns = [ICNN3(dimx, k, depth, symm_act_first=symm_act_first, softplus_type=softplus_type, zero_softplus=zero_softplus) for _ in range(nblocks)]
# icnns = [ICNN(dimx, k, depth) for _ in range(nblocks)]
layers = [None] * (nblocks + 1)
layers[0] = ActNorm(dimx)
layers[1:] = [DeepConvexFlow(icnn, bias_w1=-0.0, trainable_w0=False) for _, icnn in zip(range(nblocks), icnns)]

IDENTICAL = SequentialFlow(layers)
flow = SequentialFlow(layers)

In [3]:

def Identity(phi, coor):
    return torch.mean(torch.sum((phi - coor)**2, dim = 1, keepdim = True))

def PorusPretrain(model):
    x = torch.linspace(-3, 3, 101)
    y = torch.linspace(-3, 3, 101)
    X, Y = torch.meshgrid(x, y)
    coor = torch.cat((X.flatten()[:, None], Y.flatten()[:, None]), dim=1)

    optimizer = torch.optim.LBFGS(model.parameters(),
                            history_size=100,
                            max_iter = 100,
                            line_search_fn= 'strong_wolfe')
    def closure():
        optimizer.zero_grad()
        phi, lgdet = model.forward_transform(coor)
        loss = Identity(phi, coor)
        loss.backward()
        return loss

    optimizer.step(closure)

PorusPretrain(IDENTICAL)

new, lgd = IDENTICAL.forward_transform(coor)
flow.load_state_dict(IDENTICAL.state_dict())
print(Identity(new, coor))
torch.save(IDENTICAL.state_dict(), './data/IDENTICAL.pt')

/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


tensor(1.9304e-09, grad_fn=<MeanBackward0>)


In [ ]:
abs = []
rel = []

for epoch in range(100):
    t = 0.1 + 0.005*(epoch + 1)
    optimizer = torch.optim.LBFGS(flow.parameters(), 
                            history_size = 80,
                            max_iter = 80,
                            line_search_fn= 'strong_wolfe')
    def closure():
        optimizer.zero_grad()
        phi, lgdet = flow.forward_transform(coor)
        lgdet.unsqueeze_(-1)
        loss = Porus(phi, coor, rho, lgdet, alpha)[0]
        loss.backward()
        return loss

    optimizer.step(closure)

    phi, lgdet = flow.forward_transform(coor)
    coor = phi
    coor = coor.detach()
    lgdet = lgdet.unsqueeze_(-1).detach()
    rho = rho / (torch.exp(lgdet) + 1e-8)

    torch.save(coor, f'./data/coor{epoch + 1}.pt')
    torch.save(rho, f'./data/rho{epoch + 1}.pt')

    act_val = Barenblatt(coor, t)
    err = torch.mean((rho - act_val)**2)
    relerr = err/torch.mean(act_val**2)

    abs.append(err.item())
    rel.append(relerr.item())


    print(f'in epoch {epoch}, the L2 relative error is {relerr.item()}')
    DensityPlot(coor.numpy()[:,0], coor.numpy()[:,1], rho.numpy(), epoch+1)

    flow.load_state_dict(IDENTICAL.state_dict())

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5))
# ax.set_title(r'$f = 2\sin{x}\cdot\cos{y}$')
interval = list(range(1,61))
interval = [0.01*ele for ele in interval]
# val = [abs(ele) for ele in ENERGY]
# lines = []
ax.set_yscale('log')
lines = ax.plot(interval, rel[:60],  color= '#EE82EE' )
ax.grid(color='grey', linestyle='-', linewidth=0.25, alpha=0.5)
ax.set_xlabel('time', fontsize = 20)
ax.set_ylabel('L2 Relative Eroor', fontsize = 20)
ax.tick_params(axis='both', which='major', labelsize=20)
plt.xlim([-0.01, 0.61])
plt.savefig('./L2Relative.png',pad_inches = 0.05, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5))
# ax.set_title(r'$f = 2\sin{x}\cdot\cos{y}$')
interval = list(range(1,61))
interval = [0.01*ele for ele in interval]
# val = [abs(ele) for ele in ENERGY]
# lines = []
ax.set_yscale('log')
lines = ax.plot(interval, abs[:60],  color= '#EE82EE' )
ax.grid(color='grey', linestyle='-', linewidth=0.25, alpha=0.5)
ax.set_xlabel('time', fontsize = 20)
ax.set_ylabel('L2 Absolute Eroor', fontsize = 20)
ax.tick_params(axis='both', which='major', labelsize=20)
plt.xlim([-0.01, 0.61])
plt.savefig('./L2Absolute.png',pad_inches = 0.05, bbox_inches='tight')
plt.show()
plt.close()

In [9]:
torch.save(abs, './data/PorusAbsError.pt')
torch.save(rel, './data/PorusRelError.pt')

In [ ]:
!zip -r /content/plot.zip /content/plots/
!zip -r /content/data.zip /content/data/

In [15]:
from google.colab import files
files.download("/content/data.zip")
files.download("/content/plot.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>